HEADER

Required installed python packages:
- ```pandas```
- ```numpy```
- ```ase```
- ```matplotlib```
- ```scipy```

the functions called in the examples are inside ```lib.py``` library file and are import with "```from lib import```" instance.

Examples of the functions that read frequencies from different files (ORCA.hess, g98.out, table.csv) and create a numpy 1D array:

In [ ]:
from lib import read_freq_gaussian, read_freq_orca, read_freq_from_csv

freq_h2o = read_freq_orca('data/H2O/h2o.hess')
print('H2O freq:   ' + str(freq_h2o))
freq_nh3 = read_freq_gaussian('data/NH3/nh3.log')
print('NH3 freq:   ' + str(freq_nh3))
freq_ch3oh = read_freq_from_csv('data/freq_methanol_periodic.csv','mol_iso_cry')
print('CH3OH freq: ' + str(freq_ch3oh))

Examples of reading ```.xyz``` file with ASE and computed mass and inertia moments:

In [ ]:
from ase.io import read

methanol = read('data/CH3OH/ch3oh.xyz')
mass	 = methanol.get_masses().sum()
print('Methanol mass [AMU]: ' + str(mass))
inertia_moments = methanol.get_moments_of_inertia()
print('Methanol inertia moments [AMU*A^2]: ' + str(inertia_moments))

Example of ZPE correction and Thermal correction to the Binding Energy:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lib import read_freq_from_csv, BH_t, ZPE

### PARAMETERS and INITIALIZATION ###
step 			= 10	#Temperature steps
t_in 			= 10	#Initial Temperature
t_fin			= 310	#Final Temperature
T 				= np.arange(t_in, t_fin, step,dtype=float) #1D numpy array of Temperature values
tmp_correction	= np.arange(t_in, t_fin, step,dtype=float) #1D numpy array of correction to BE Temperature values

### INPUT INFORMATION ###
ch3oh_be			= 70.1965 #BE without ZPE correction
ch3oh_csv_path		= 'data/freq_methanol_periodic.csv'

### READ FREQUENCIES ###
freq_isolated_mol 	= read_freq_from_csv(ch3oh_csv_path,'mol_iso_cry')
freq_isolated_surf 	= read_freq_from_csv(ch3oh_csv_path,'CH3OH_cry_ref')
freq_complex	 	= read_freq_from_csv(ch3oh_csv_path,'CH3OH_cry_complex')

### ZPE correction to BE ###
zpe_BE 			= ZPE(freq_isolated_mol) + ZPE(freq_isolated_surf) - ZPE(freq_complex)
print('ZPE correction to BE: ' + str(zpe_BE))
BH_0 			= ch3oh_be + zpe_BE
print('BH(0):                 ' + str(BH_0))

### COMPUTE THE THERMAL CORRECTION TO THE BE ###
for i in range(len(T)):
	tmp_correction[i]	= BH_t(T[i],freq_isolated_mol,freq_isolated_surf,freq_complex,False)

### SAVE THERMAL CORRECTION and BH(T) in .csv file ####
df_tmp = pd.DataFrame()
df_tmp['T'] 			= T
df_tmp['Delta_BH_t']	= tmp_correction
BH_T 					= tmp_correction + BH_0
df_tmp['BH_t']			= BH_T
with open("tmp/BH_t.csv","w+") as output:
	output.write(df_tmp.to_csv(sep="\t", index=False))

### PLOT THE THERMAL CORRECTION ####
fig, ax = plt.subplots(figsize=(7,5))
ax.plot(T, tmp_correction,color='magenta',linewidth=5,label='Thermal correction')
#ax.plot(T, BH_T,color='orange',linewidth=5,label='BH(T)')
ax.set_xlabel('T [K]', fontsize=15)
ax.set_ylabel('BH(T)-BH(0) kJ/mol', fontsize=15)
ax.yaxis.set_tick_params(labelsize=15)
ax.xaxis.set_tick_params(labelsize=15)
ax.legend(fontsize=15)
plt.title('BE Thermal corretion')
ax.grid()
plt.show()

Convert pandas reaction network dataframe to graph raction network with species and reaction as different type of nodes

- path_to_molecules_database 		--> path to the molecules database
- path_to_network					--> path to the network to be clean
- threshold_endothermicity_kJmol	--> threshold to determined the reactions to be deleted for endothermicity (in kJ/mol)